In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
import pandas as pd
import tensorflow as tf

# Import our input dataset
all_start_df = pd.read_csv("../Data/Clean_Data/clean_start_mlb.csv", sep=",", encoding='latin-1')
all_start_df.head()

,playerID,sport,position,start_year,start_age,years_played
0,aardsda01,mlb,G_p,2004,23,12
1,abadan01,mlb,G_if,2001,29,5
2,abadan01,mlb,G_p,2001,29,5
3,abbotje01,mlb,G_of,1997,25,5
4,abbotku01,mlb,G_of,1993,24,8


In [2]:
del all_start_df['playerID']
del all_start_df['sport']
all_start_df.head()

,position,start_year,start_age,years_played
0,G_p,2004,23,12
1,G_if,2001,29,5
2,G_p,2001,29,5
3,G_of,1997,25,5
4,G_of,1993,24,8


In [3]:
# Generate our categorical variable list
start_cat = all_start_df.dtypes[all_start_df.dtypes == "object"].index.tolist()


# Check the number of unique values in each column
all_start_df[start_cat].nunique()

position    4
dtype: int64

In [4]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(all_start_df[start_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(start_cat)
encode_df.head()

,position_G_c,position_G_if,position_G_of,position_G_p
0,0.0,0.0,0.0,1.0
1,0.0,1.0,0.0,0.0
2,0.0,0.0,0.0,1.0
3,0.0,0.0,1.0,0.0
4,0.0,0.0,1.0,0.0


In [5]:
# Merge one-hot encoded features and drop the originals
all_start_df = all_start_df.merge(encode_df,left_index=True, right_index=True)
all_start_df = all_start_df.drop(start_cat,1)
all_start_df.head()

,start_year,start_age,years_played,position_G_c,position_G_if,position_G_of,position_G_p
0,2004,23,12,0.0,0.0,0.0,1.0
1,2001,29,5,0.0,1.0,0.0,0.0
2,2001,29,5,0.0,0.0,0.0,1.0
3,1997,25,5,0.0,0.0,1.0,0.0
4,1993,24,8,0.0,0.0,1.0,0.0


In [6]:
# Remove years_played target from features data
y = all_start_df.years_played.values
X = all_start_df.drop(columns=["years_played"]).values

# Split training/test datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [7]:
# Create the SVM model
svm = SVC(kernel='linear')

# Train the model
svm.fit(X_train, y_train)

# Evaluate the model
y_pred = svm.predict(X_test_scaled)
print(f" SVM model accuracy: {accuracy_score(y_test,y_pred):.3f}")

 SVM model accuracy: 0.025


In [10]:
# Define the model - deep neural net
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 =  64
hidden_nodes_layer2 = 64

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))


# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the Sequential model together and customize metrics
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [11]:
# Train the model 
fit_model = nn.fit(X_train_scaled, y_train, epochs=50) 
# Evaluate the model using the test data 
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Epoch 1/50
127/127 [==============================] - 0s 500us/step - loss: -67.1058 - accuracy: 0.2201
Epoch 2/50
127/127 [==============================] - 0s 485us/step - loss: -984.2059 - accuracy: 0.2211
Epoch 3/50
127/127 [==============================] - 0s 493us/step - loss: -4689.0425 - accuracy: 0.2211
Epoch 4/50
127/127 [==============================] - 0s 485us/step - loss: -13477.6455 - accuracy: 0.2211
Epoch 5/50
127/127 [==============================] - 0s 477us/step - loss: -29169.6816 - accuracy: 0.2211
Epoch 6/50
127/127 [==============================] - 0s 493us/step - loss: -53592.3945 - accuracy: 0.2211
Epoch 7/50
127/127 [==============================] - 0s 485us/step - loss: -87536.2344 - accuracy: 0.2211
Epoch 8/50
127/127 [==============================] - 0s 493us/step - loss: -132560.4844 - accuracy: 0.2211
Epoch 9/50
127/127 [==============================] - 0s 485us/step - loss: -190060.8438 - accuracy: 0.2211
Epoch 10/50
127/127 [====================